<a href="https://colab.research.google.com/github/CamCranda11/MLFA25Project/blob/main/MLProjectModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

song_data = pd.read_csv('spotify_data.csv')

print("--- Original Data ---")
print(song_data.head())
print("\n")

--- Original Data ---
   Unnamed: 0    artist_name        track_name                track_id  \
0           0     Jason Mraz   I Won't Give Up  53QF56cjZA9RTuuMZDrSA6   
1           1     Jason Mraz  93 Million Miles  1s8tP3jP4GZcyHDsjvw218   
2           2  Joshua Hyslop  Do Not Let Me Go  7BRCa8MPiyuvr2VU3O9W0F   
3           3   Boyce Avenue          Fast Car  63wsZUhUZLlh1OsyrZq7sz   
4           4   Andrew Belle  Sky's Still Blue  6nXIYClvJAfi6ujLiKqEq8   

   popularity  year     genre  danceability  energy  key  loudness  mode  \
0          68  2012  acoustic         0.483   0.303    4   -10.058     1   
1          50  2012  acoustic         0.572   0.454    3   -10.286     1   
2          57  2012  acoustic         0.409   0.234    3   -13.711     1   
3          58  2012  acoustic         0.392   0.251   10    -9.845     1   
4          54  2012  acoustic         0.430   0.791    6    -5.419     0   

   speechiness  acousticness  instrumentalness  liveness  valence    tempo  

In [43]:
features_to_cluster = ['danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence']
features_df = song_data[features_to_cluster].copy()

mode_value = features_df['mode'].mode()[0]
features_df['mode'] = features_df['mode'].fillna(mode_value)

features_df = features_df.fillna(features_df.mean())

scaler = StandardScaler()
scaled_features_df = scaler.fit_transform(features_df)

print("--- Scaled Features (First 5 Rows) ---")
print(scaled_features_df[:5])
print("\n")

--- Scaled Features (First 5 Rows) ---
[[-0.29509342 -1.24461718  0.75872495 -0.39352278  1.04922991 -0.69122871
  -1.17892497]
 [ 0.18734904 -0.68639325  0.75872495 -0.52833738  0.43794003 -0.69119118
   0.22134908]
 [-0.69622536 -1.49969964  0.75872495 -0.47709206  0.04637646 -0.69109175
  -1.15658017]
 [-0.78837729 -1.43685324  0.75872495 -0.44555648  1.36755137 -0.69122871
   0.19528015]
 [-0.58239062  0.55944426 -1.31800069 -0.49364824 -0.70125643 -0.63836256
  -0.88844259]]




In [44]:
def recommend_songs(input_song_name, input_artist_name, data_df, clustering_approach='global', num_recs=3, features_to_cluster=None, optimal_k=50):
    try:
        song_row = data_df[
            (data_df['track_name'] == input_song_name) &
            (data_df['artist_name'] == input_artist_name)
        ]

        if song_row.empty:
             return f"Song '{input_song_name}' by {input_artist_name} not found in the dataset."

        song_row = song_row.iloc[0]
        song_genre = song_row['genre']

        if clustering_approach == 'global':
            if 'cluster_id' not in data_df.columns:
                 return "Global clustering approach requires 'cluster_id' column in the dataframe."
            song_cluster = song_row['cluster_id']
            recommendations = data_df[
                (data_df['cluster_id'] == song_cluster) &
                (data_df['genre'] == song_genre) &
                ((data_df['track_name'] != input_song_name) | (data_df['artist_name'] != input_artist_name))
            ]

        elif clustering_approach == 'genre':
            if features_to_cluster is None or optimal_k is None:
                 return "Genre clustering approach requires 'features_to_cluster' and 'optimal_k'."

            genre_df = data_df[data_df['genre'] == song_genre].copy()

            if len(genre_df) < optimal_k:
                 return f"Not enough songs ({len(genre_df)}) in the genre '{song_genre}' to perform clustering with k={optimal_k}."

            genre_features_df = genre_df[features_to_cluster].copy()
            mode_value = genre_features_df['mode'].mode()[0]
            genre_features_df['mode'] = genre_features_df['mode'].fillna(mode_value)
            genre_features_df = genre_features_df.fillna(genre_features_df.mean())

            genre_scaler = StandardScaler()
            scaled_genre_features_df = genre_scaler.fit_transform(genre_features_df)

            genre_kmeans_model = KMeans(n_clusters=optimal_k, init='k-means++', n_init=10, random_state=42)
            genre_cluster_labels = genre_kmeans_model.fit_predict(scaled_genre_features_df)

            genre_df_clustered = genre_df.copy()
            genre_df_clustered['genre_cluster_id'] = genre_cluster_labels

            input_song_genre_cluster = genre_df_clustered[
                (genre_df_clustered['track_name'] == input_song_name) &
                (genre_df_clustered['artist_name'] == input_artist_name)
            ].iloc[0]['genre_cluster_id']

            recommendations = genre_df_clustered[
                (genre_df_clustered['genre_cluster_id'] == input_song_genre_cluster) &
                ((genre_df_clustered['track_name'] != input_song_name) | (genre_df_clustered['artist_name'] != input_artist_name))
            ]
        else:
            return "Invalid clustering_approach. Use 'global' or 'genre'."


        if len(recommendations) == 0:
            return f"No similar songs found in the same {'genre and cluster' if clustering_approach == 'genre' else 'cluster and genre'}."

        return recommendations.sample(min(num_recs, len(recommendations)))

    except IndexError:
        return f"Song '{input_song_name}' by {input_artist_name} not found in the dataset."
    except Exception as e:
        return f"An error occurred: {e}"

In [56]:
my_song = 'King Kunta'
my_artist = 'Kendrick Lamar'

display_columns = ['track_name', 'artist_name', 'track_id', 'genre', 'danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence']

input_song_info = song_data[
    (song_data['track_name'] == my_song) &
    (song_data['artist_name'] == my_artist)
]
print(f"--- Input Song: '{my_song}' by {my_artist} ---")
if not input_song_info.empty:
    display(input_song_info[display_columns])
else:
    print("Input song not found in the dataset.")

recommendations_genre = recommend_songs(
    my_song,
    my_artist,
    song_data,
    clustering_approach='genre',
    num_recs=5,
    features_to_cluster=['danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence'],
    optimal_k=50
)

print(f"\n--- Recommendations based on '{my_song}' by {my_artist} ---")
if isinstance(recommendations_genre, pd.DataFrame):
    rec_display_columns = display_columns + ['genre_cluster_id'] if 'genre_cluster_id' in recommendations_genre.columns else display_columns
    display(recommendations_genre[rec_display_columns])
else:
    print(recommendations_genre)

--- Input Song: 'King Kunta' by Kendrick Lamar ---


,track_name,artist_name,track_id,genre,danceability,energy,mode,speechiness,acousticness,instrumentalness,valence
188807,King Kunta,Kendrick Lamar,0N3W5peJUQtI4eyR6GJT5O,hip-hop,0.884,0.657,0,0.0977,0.00589,0.0,0.489



--- Recommendations based on 'King Kunta' by Kendrick Lamar ---


,track_name,artist_name,track_id,genre,danceability,energy,mode,speechiness,acousticness,instrumentalness,valence,genre_cluster_id
560984,Rax,Lil Wayne,13C7ucXlnNRWcZWzgvSbP4,hip-hop,0.744,0.599,0,0.2270,0.04820,0.000000,0.525,5
341057,No Time (feat. Gunna),Playboi Carti,5pHJv0bgNsT9nPoK2BjNBn,hip-hop,0.943,0.763,0,0.1580,0.01750,0.000018,0.225,5
1047499,Thug,Slim Thug,00mojm5rDbk8dhvwHw4IIz,hip-hop,0.784,0.712,0,0.0923,0.00874,0.001690,0.313,5
139531,Lac Lac,Big K.R.I.T.,4XRhTVExI4QKkkZHojtzzb,hip-hop,0.803,0.652,0,0.1530,0.16000,0.000000,0.530,5
397954,Goin Dummi,Future,6QFzUXTIZXOLesQcgmGOsR,hip-hop,0.888,0.665,0,0.0930,0.01510,0.000000,0.568,5


In [46]:
import pandas as pd

artist_name = "Joywave"

pd.set_option('display.max_rows', None)

artist_songs = song_data[song_data['artist_name'] == artist_name]

if len(artist_songs) > 0:
    print(f"--- Songs by {artist_name} ---")
    display(artist_songs)
else:
    print(f"No songs found for artist: {artist_name}")

pd.reset_option('display.max_rows')

--- Songs by Joywave ---


,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
2654,2654,Joywave,Golden State,6b6KGKBboxwTgTxtYZYE7I,25,2012,alt-rock,0.483,0.919,6,-5.093,1,0.1050,0.013200,0.002960,0.1090,0.3490,137.932,224667,4
34104,34104,Joywave,True Grit,1H8EZ7uCaKBEX97B7KoHTI,24,2012,indie-pop,0.517,0.945,11,-5.512,0,0.0454,0.058900,0.002600,0.3180,0.3070,127.945,245907,4
34115,34115,Joywave,Anemone,0geaQiAZN8AhkxLfyAiUXe,24,2012,indie-pop,0.608,0.872,5,-5.872,0,0.0404,0.153000,0.001080,0.0507,0.3230,106.981,255933,4
162971,162971,Joywave,Destruction,2iLxXSM7AOzB4RCNzk4bjd,53,2015,alt-rock,0.660,0.638,0,-6.827,1,0.0379,0.000438,0.019100,0.1070,0.5580,108.004,184413,4
162989,162989,Joywave,Tongues,5mCprFWOqe0jB96v9RhLBo,52,2015,alt-rock,0.827,0.682,6,-7.111,0,0.0435,0.000480,0.018700,0.1010,0.9330,117.967,234773,4
163151,163151,Joywave,Somebody New,2vcfImyMOQjYhnKOVkjoIR,42,2015,alt-rock,0.456,0.940,3,-3.334,0,0.0703,0.000049,0.000165,0.3280,0.4630,174.038,206320,4
163220,163220,Joywave,Traveling at the Speed of Light,5zZOhHw2p7VLPau3A29hJw,39,2015,alt-rock,0.570,0.636,0,-7.161,0,0.0418,0.315000,0.004280,0.0784,0.3640,135.010,453600,4
163239,163239,Joywave,Nice House,0GAvMukpDYssdZgMLsSuhx,38,2015,alt-rock,0.549,0.754,5,-6.818,1,0.0489,0.188000,0.069200,0.1240,0.0475,134.988,340040,4
163358,163358,Joywave,Now,2UbzSDEYP8eCg5K2a8d3PM,35,2015,alt-rock,0.575,0.872,1,-6.259,1,0.0854,0.004090,0.004690,0.0944,0.2740,125.994,262213,4
163542,163542,Joywave,Bad Dreams,0B7B8DAnYcWHqcwqOpenth,32,2015,alt-rock,0.733,0.327,5,-7.324,0,0.0479,0.132000,0.000592,0.1320,0.4060,118.058,256440,4
